# SHAPXplain: Example Usage

This notebook demonstrates how to use the SHAPXplain package to integrate SHAP explanations with LLMs.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from shap import TreeExplainer
from shapxplain import ShapLLMExplainer

# Load data and train model
data = load_iris()
X, y = data.data, data.target
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Generate SHAP values
explainer = TreeExplainer(model)
shap_values = explainer.shap_values(X)


In [ ]:
llm_explainer = ShapLLMExplainer(
    model=model,
    feature_names=data.feature_names,  # Add feature names
)

# Generate an explanation for a specific data point
explanation = llm_explainer.explain(
    shap_values=shap_values[0],
    data_point=X[0],
    prediction=model.predict_proba(X[0].reshape(1, -1))[0],
    prediction_class=data.target_names[model.predict(X[0].reshape(1, -1))[0]]
)